Finite Difference form of the inhomgeneous wave equation:

\begin{equation}
p_{i,j}^{n+1}=2p_{i,j}^n-p_{i,j}^{n-1}+\\
\frac{(\Delta t)^2}{(\Delta x)^2} \left[ c_{i,j}^2 \left( p_{i-1,j}+p_{i+1,j}+p_{i,j-1}+p_{i,j+1}-4p_{i,j} \right)-
\frac{1}{4\rho_{i,j}} \left( \left(\rho_{i+1,j}-\rho_{i-1,j}\right)\left(p_{i+1,j}-p_{i-1,j}\right) + 
\left(\rho_{i,j+1}-\rho_{i,j-1}\right)\left(p_{i,j+1}-p_{i,j-1}\right)\right)
\right]
\end{equation}

In [76]:
%matplotlib inline
import math as m
import numpy #array operations
import matplotlib.pyplot as plt #plotting
from matplotlib.collections import LineCollection
import matplotlib.colors as colors
from IPython import display #for continous display
from scipy import weave
from scipy.weave import converters
import time

def stepccode(u,un,unn,rho,cc,cconst,nx,ny):
    code = """
            for (int i=1; i<nx-1; ++i) {
                for (int j=1; j<ny-1; ++j) {
                    u(i,j) = 2.*un(i,j)-unn(i,j)+cconst*(cc(i,j)*(un(i-1,j)+un(i+1,j)+un(i,j-1)+un(i,j+1)-4.*un(i,j))-
                            -0.25/rho(i,j)*((rho(i+1,j)-rho(i+1,j))*(un(i+1,j)-un(i-1,j))+
                                            (rho(i,j+1)-rho(i,j-1))*(un(i,j+1)-un(i,j-1))));
                }
            }
            """
    weave.inline(code,['u', 'un', 'unn', 'rho', 'cc', 'cconst','nx','ny'],
                   type_converters = converters.blitz)#, compiler = 'gcc')
    return u

def stepblitz(u,un,unn,rho,cc,cconst):
    
    expr="u[1:-1,1:-1]=2.*un[1:-1,1:-1]-unn[1:-1,1:-1]+cconst*(cc[1:-1,1:-1]*\
        (un[:-2,1:-1]+un[2:,1:-1]+un[1:-1,:-2]+un[1:-1,2:]-4.*un[1:-1,1:-1])-\
        .25*rho[1:-1,1:-1]*((rho[2:,1:-1]-rho[:-2,1:-1])*(un[2:,1:-1]-un[:-2,1:-1])+\
        (rho[1:-1,2:]-rho[1:-1,:-2])*(un[1:-1,2:]-un[1:-1,:-2])))"
    weave.blitz(expr, check_size=0)
    
    return u

def plotwave(u,time):
    #plt.figure(1)
    plt.clf()
    #plot the pressue field
    plt.imshow(u, origin='upper', extent=[0., 2., 0., 2.], vmax=2, vmin=-2) #plot the wave field
    plt.text(0.1,1.8,"time {0:.5f}".format(time)) #annotate the time
    plt.gca().set_xlim([0.,2.])
    plt.gca().set_ylim([0.,2.])
    display.clear_output(wait=True)
    display.display(plt.gcf())
    
def solvewave(plot=True):
    #computational domain
    nx = ny = 128
    size=2. #size of the domain
    #parameters of the wave
    c = 1. #speed of sound of homogeneous me`dium 
    l=.3 #wavelength
    nu=c/l #frequency
    omega=nu*2.*m.pi #angular frequency
    duration=50./nu #duration of source
    
    #position of source
    emissionlength=(100./100.)*nx #in percent
    startx=int(nx/2-emissionlength/2)
    endx=int(nx/2+emissionlength/2)

    #further variables
    dx = size/(nx-1)
    CFL=0.05#CFL number < 1/sqrt(2)
    dt = CFL*dx/c 
    nt=int(2./dt) #number of time steps

    sourcepos=0

    #every xx times over the total nt timesteps an output should be generated 
    output=map(int,list(numpy.linspace(1,nt,int(nt*float(5.5/100.)))))

    u  = numpy.zeros((nx,ny)) #pressure at t
    un = numpy.zeros((nx,ny)) #pressure at t-dt
    unn= numpy.zeros((nx,ny)) #pressure at t-2*dt

    #homogeneous speed of sound c**2 !
    cc = numpy.ones((nx,ny))*1.
    #homogeneous density
    rho = numpy.ones((nx,ny))*1.
    
    #build the double slit
    xx,yy=numpy.meshgrid(numpy.linspace(0,2.,nx),numpy.linspace(0,2.,nx))
    ow=0.1
    dw=1.
    obstacle=((xx>.3) & (yy>1.+dw/2.+ow/2.) & (xx<.35))|\
        ((xx>.3) & (yy<1.-dw/2.-ow/2.) & (xx<0.35))|\
        ((xx>.3) & (yy>1.-dw/2.+ow/2.) & (yy<1.+dw/2.-ow/2.)& (xx<0.35))
    a=numpy.where(obstacle)
    cc[a]=5.**2.
    rho[a]=5.    
    cconst=dt*dt/dx/dx
    
    plt.figure(1, figsize=(8, 8), dpi=300)

    for n in range(nt+1): ##loop across number of time steps

        #here we compute the wave propagation in an inhomogeneous media
        #using blitz to speed up things
       
        
        #u=stepblitz(u,un,unn,rho,cc,cconst)
        u=stepccode(u,un,unn,rho,cc,cconst,nx,ny)
        
        #Impose the boundary conditions                 
        #north
        u[0,:] = -dx/dt/(cc[0,:]**.5)*(u[1,:]-un[1,:])+u[1,:]
        #u[0,:] = u[1,:]
        #south
        u[-1,:] = -dx/dt/(cc[-1,:]**.5)*(u[-2,:]-un[-2,:])+u[-2,:]
        #u[-1,:] = u[-2,:]
        #west
        u[:,0] = -dx/dt/(cc[:,0]**.5)*(u[:,1]-un[:,1])+u[:,1]
        #u[:,0] = u[:,1]
        #east
        u[:,-1] = -dx/dt/(cc[:,-1]**.5)*(u[:,-2]-un[:,-2])+u[:,-2]
        #u[:,-1] = u[:,-2]
        
        #pressure source
        if n*dt<(duration+10*dt):
            #generate a source that goes to zero after duration 
            usource=numpy.sin(omega*(n*dt))*(n*dt<duration)
            u[startx:endx,sourcepos]=usource
        
        #save values for the time derivative 
        unn=un.copy() #n-1 time steop
        un=u.copy()   #n time step
        
        if (n in output) & plot:
            uplot=u.copy()
            uplot[a]=0.
            plotwave(uplot,n*dt)
            
if __name__ == "__main__":
    # execute only if run as a script
    t = time.clock()
    solvewave(False)
    print time.clock() - t

0.798175


In [48]:
xx=numpy.linspace(0,2.,10)

In [ ]:
(xx>.5) & (xx<.9)